<a href="https://colab.research.google.com/github/sifatbhuiyan0909/Dhaka-Finance-Navigator/blob/main/notebooks/05_missing_data_handling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

***Bringing the data from the local machine***

In [1]:
from google.colab import files
uploaded= files.upload()

Saving Dhaka-Stock-Exchange-DSE-2021.csv to Dhaka-Stock-Exchange-DSE-2021.csv
Saving Dhaka-Stock-Exchange-DSE-2020.csv to Dhaka-Stock-Exchange-DSE-2020.csv
Saving Dhaka-Stock-Exchange-DSE-2019.csv to Dhaka-Stock-Exchange-DSE-2019.csv
Saving Dhaka-Stock-Exchange-DSE-2018.csv to Dhaka-Stock-Exchange-DSE-2018.csv
Saving Dhaka-Stock-Exchange-DSE-2017.csv to Dhaka-Stock-Exchange-DSE-2017.csv
Saving Dhaka-Stock-Exchange-DSE-2016.csv to Dhaka-Stock-Exchange-DSE-2016.csv
Saving Dhaka-Stock-Exchange-DSE-2015.csv to Dhaka-Stock-Exchange-DSE-2015.csv
Saving Dhaka-Stock-Exchange-DSE-2014.csv to Dhaka-Stock-Exchange-DSE-2014.csv
Saving Dhaka-Stock-Exchange-DSE-2013.csv to Dhaka-Stock-Exchange-DSE-2013.csv
Saving Dhaka-Stock-Exchange-DSE-2012.csv to Dhaka-Stock-Exchange-DSE-2012.csv
Saving Dhaka-Stock-Exchange-DSE-2011.csv to Dhaka-Stock-Exchange-DSE-2011.csv
Saving Dhaka-Stock-Exchange-DSE-2010.csv to Dhaka-Stock-Exchange-DSE-2010.csv
Saving Dhaka-Stock-Exchange-DSE-2009.csv to Dhaka-Stock-Exchange

In [2]:
import pandas as pd
import glob
import os

# --- 1. Data Acquisition (Day 3) ---
# Finds all CSV files and merges them into master_df
all_filenames = glob.glob(os.path.join('.', '*.csv'))

# pd.read_csv(header=None) loads files without using the first row as names
master_df = pd.concat([pd.read_csv(f, header=None) for f in all_filenames], ignore_index=True)

# Rename the columns
master_df.columns = ['Ticker', 'Date', 'Open', 'High', 'Low', 'Close', 'Volume']

# --- 2. Structural & Type Cleaning (Day 4) ---
# Create a copy for cleaning
df = master_df.copy()

# Converting Date to datetime format and set it as the index
df['Date'] = pd.to_datetime(df['Date'], format='%d-%m-%Y')
df = df.set_index('Date')

# Clean and convert the numerical columns: Open, Low, Volume
for col in ['Open', 'Low', 'Volume']:
    # Remove the problematic dash/hyphen
    df[col] = df[col].astype(str).str.replace('-', '', regex=False)

    # Convert to numeric, forcing bad data (like remnants of 'n') to NaN
    df[col] = pd.to_numeric(df[col], errors='coerce')

print("--- DataFrame Info After Setup ---")
df.info()

# --- 3. Day 5 Task: Quantify Missing Values ---
print("\n--- NaN Counts Per Column (The Challenge for Day 5) ---")
# .isnull() finds all NaNs, and .sum() counts them by column
print(df.isnull().sum())

--- DataFrame Info After Setup ---
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1198363 entries, 2002-01-01 to 2003-12-30
Data columns (total 6 columns):
 #   Column  Non-Null Count    Dtype  
---  ------  --------------    -----  
 0   Ticker  1198363 non-null  object 
 1   Open    1198361 non-null  float64
 2   High    1198362 non-null  float64
 3   Low     1198360 non-null  float64
 4   Close   1198362 non-null  float64
 5   Volume  1198361 non-null  float64
dtypes: float64(5), object(1)
memory usage: 64.0+ MB

--- NaN Counts Per Column (The Challenge for Day 5) ---
Ticker    0
Open      2
High      1
Low       3
Close     1
Volume    2
dtype: int64


In [3]:
print(df.isnull().sum())

Ticker    0
Open      2
High      1
Low       3
Close     1
Volume    2
dtype: int64


In [4]:
# Selecting the columns where we found NaNs (price and volume):
cols_to_fill = ['Open', 'High', 'Low', 'Close', 'Volume']

print("--- Dataframe shape BEFORE filling NaNs ---")
print(df.shape)

# Applying forward-fill imputation to all selected columns:
df[cols_to_fill] = df[cols_to_fill].ffill()

print("\n--- NaN Check AFTER filling ---")
print(df.isnull().sum())
print("\n--- Dataframe shape AFTER filling NaNs ---")
print(df.shape)

--- Dataframe shape BEFORE filling NaNs ---
(1198363, 6)

--- NaN Check AFTER filling ---
Ticker    0
Open      0
High      0
Low       0
Close     0
Volume    0
dtype: int64

--- Dataframe shape AFTER filling NaNs ---
(1198363, 6)
